# First Iteration Code Development and Testing

## Algorithmic Trading - Applying Moving Average

### Importing Libraries and Dependencies

In [6]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import warnings
warnings.simplefilter(action="ignore")

In [7]:
# Input Parameters

# Window runs: Short Term (9-21, 21-50, 9-50)
# Long Term: 50-100, 21-100, Golden Cross - Death Cross: 50-200
# Objective: Which of the above combinations gives the best return for each stock

# Set the short window and long windows
short_window = 9
long_window = 21

# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

### Importing Source CSV as Dataframe

In [8]:
# Reading CSV
# Set the file path
filepath = Path("../Resources/AAPL.csv")

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index and auto-format the datetime string
aapl_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
aapl_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-07,37.174999,37.207500,36.474998,36.982498,36.123104,219111200
1,2019-01-08,37.389999,37.955002,37.130001,37.687500,36.811718,164101200
2,2019-01-09,37.822498,38.632500,37.407501,38.327499,37.436844,180396400
3,2019-01-10,38.125000,38.492500,37.715000,38.450001,37.556507,143122800
4,2019-01-11,38.220001,38.424999,37.877499,38.072498,37.187771,108092800


### Generating Moving Average Crossover Signal

In [9]:
# Grab just the `date` and `close` from the dataset
signals_df = aapl_df.loc[:, ["Date", "Close"]].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2019-01-07,36.982498,NaN,NaN,0.0,NaN
2019-01-08,37.687500,NaN,NaN,0.0,0.0
2019-01-09,38.327499,NaN,NaN,0.0,0.0
2019-01-10,38.450001,NaN,NaN,0.0,0.0
2019-01-11,38.072498,NaN,NaN,0.0,0.0
2019-01-14,37.500000,NaN,NaN,0.0,0.0
2019-01-15,38.267502,NaN,NaN,0.0,0.0
2019-01-16,38.735001,NaN,NaN,0.0,0.0
2019-01-17,38.965000,38.109722,NaN,0.0,0.0


 ### Plotting Entry/Exit Points of Moving Average Crossover Trading Strategy

In [10]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

In [11]:
# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

### Backtest the Trading Strategy

In [12]:
# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df["Position"] = share_size * signals_df["Signal"]

# Find the points in time where a 500 share position is bought or sold
signals_df["Entry/Exit Position"] = signals_df["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df["Portfolio Holdings"] = (
    signals_df["Close"] * signals_df["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df["Portfolio Cash"] = (
    initial_capital - (signals_df["Close"] * signals_df["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df["Portfolio Total"] = (
    signals_df["Portfolio Cash"] + signals_df["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
signals_df["Portfolio Daily Returns"] = signals_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
signals_df["Portfolio Cumulative Returns"] = (
    1 + signals_df["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
signals_df.head()

,Close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2019-01-07,36.982498,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,37.687500,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,NaN,NaN
2019-01-09,38.327499,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2019-01-10,38.450001,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2019-01-11,38.072498,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0


### Plot Entry/Exit Points of Backtest Results

In [13]:
# Visualize exit position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

### Prepare Portfolio Evaluation Metrics DataFrame

In [14]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [15]:
### Calculate and Assign Portfolio Evaluation Metrics

In [16]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']]
sortino_ratio_df.loc[:,'Downside Returns'] = 0
sortino_ratio_df.head()

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

,Backtest
Annual Return,0.177517
Cumulative Returns,0.411875
Annual Volatility,0.0929757
Sharpe Ratio,1.90928
Sortino Ratio,2.94192


### Prepare Trade Evaluation Metrics DataFrame

In [17]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [18]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss =  exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2019-02-05,2019-05-14,500.0,43.544998,47.165001,21772.4990,23582.5005,1810.0015
1,AAPL,2019-06-12,2019-08-07,500.0,48.547501,49.759998,24273.7505,24879.9990,606.2485
2,AAPL,2019-08-21,2020-02-25,500.0,53.160000,72.019997,26580.0000,36009.9985,9429.9985
3,AAPL,2020-04-08,2020-09-14,500.0,66.517502,115.360001,33258.7510,57680.0005,24421.2495
4,AAPL,2020-10-06,2020-10-28,500.0,113.160004,111.199997,56580.0020,55599.9985,-980.0035


In [19]:
### Create Hvplot Line Chart of Closing, SMA50, and SMA100

In [20]:
price_df = signals_df[['Close', 'SMA50', 'SMA100']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [21]:
### Create Hvplot Table of Portfolio Metrics

In [22]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

In [23]:
### Create Hvplot Table of Trade Metrics

In [24]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]